In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

import numpy as np
import pandas as pd

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from scikeras.wrappers import KerasClassifier

from warnings import simplefilter

2024-01-30 09:25:43.569989: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-30 09:25:43.598021: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-30 09:25:43.598062: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-30 09:25:43.598932: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-30 09:25:43.603178: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-30 09:25:43.603718: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
#with two dataset splitted
dftrain = pd.read_csv("/home/jovyan/MQTTset/train70_reduced.csv") 
dftest = pd.read_csv("/home/jovyan/MQTTset/test30_reduced.csv")

# dftrain = pd.read_csv("train70.csv", low_memory=False) 
# dftest = pd.read_csv("test30.csv", low_memory=False)

simplefilter(action='ignore', category=FutureWarning)
seed = 7

In [3]:
#train
#print(dftrain.loc[dftrain['target'] == 'legitimate'])
class_names = dftrain.target.unique()
dftrain=dftrain.astype('category')
cat_columns = dftrain.select_dtypes(['category']).columns
dftrain[cat_columns] = dftrain[cat_columns].apply(lambda x: x.cat.codes)
#print(dftrain.loc[125, 'target'])
x_columns = dftrain.columns.drop('target')
x_train = dftrain[x_columns].values
y_train = dftrain['target']

#test
class_names = dftest.target.unique()
dftest=dftest.astype('category')
cat_columns = dftest.select_dtypes(['category']).columns
dftest[cat_columns] = dftest[cat_columns].apply(lambda x: x.cat.codes)
x_columns = dftest.columns.drop('target')
x_test = dftest[x_columns].values
y_test = dftest['target']

print("Ready to generate train and test datasets")

Ready to generate train and test datasets


In [20]:
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

x_train = np.copy(x_train_scaled)
x_test = np.copy(x_test_scaled)

In [26]:
def calculate_performance_metrics(x_test, y_test, model):

    # Predictions
    y_pred = model.predict(x_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    
    # Compute metrics
    precision = metrics.precision_score(y_test, y_pred_classes, average='weighted')
    accuracy = metrics.accuracy_score(y_test, y_pred_classes)
    f1 = metrics.f1_score(y_test, y_pred_classes, average='macro')
    auc = metrics.roc_auc_score(y_test, y_pred, multi_class='ovr')

    print(f"Precision: {precision}\nAccuracy: {accuracy}\nF1 Score: {f1}\nAUC: {auc}")
    
    # Confusion matrix for FNR, TNR, FPR, TPR
    cm = metrics.confusion_matrix(y_test, y_pred_classes)
    def calculate_rates(conf_matrix, class_index):
        tp = conf_matrix[class_index, class_index]
        fn = np.sum(conf_matrix[class_index, :]) - tp
        fp = np.sum(conf_matrix[:, class_index]) - tp
        tn = np.sum(conf_matrix) - (tp + fn + fp)
    
        fnr = fn / (fn + tp)
        tnr = tn / (tn + fp)
        fpr = fp / (fp + tn)
        tpr = tp / (tp + fn)
        return fnr, tnr, fpr, tpr

    # Calculate and aggregate rates
    fnrs, tnrs, fprs, tprs = [], [], [], []
    for i in range(cm.shape[0]):
        fnr, tnr, fpr, tpr = calculate_rates(cm, i)
        fnrs.append(fnr)
        tnrs.append(tnr)
        fprs.append(fpr)
        tprs.append(tpr)
    
    mean_fnr = np.mean(fnrs)
    mean_tnr = np.mean(tnrs)
    mean_fpr = np.mean(fprs)
    mean_tpr = np.mean(tprs)

    # Printing the mean metrics
    print(f"Mean FNR: {mean_fnr}\nMean TNR: {mean_tnr}\nMean FPR: {mean_fpr}\nMean TPR: {mean_tpr}")


In [27]:
input_shape = x_train.shape[1:]  # Determined by the shape of your training data
output_shape = len(np.unique(y_train))  # Assuming classification task

# Function to create the DNN model
def create_model():
    model = Sequential()
    model.add(Dense(50, input_shape=input_shape, kernel_initializer='normal', activation='relu'))
    model.add(Dense(30, input_shape=input_shape, kernel_initializer='normal', activation='relu'))
    model.add(Dense(20, kernel_initializer='normal'))
    # model.add(Dense(10, kernel_initializer='normal'))

    model.add(Dense(output_shape, activation='softmax'))
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model


# Wrapping the model with KerasClassifier
model = KerasClassifier(model=create_model, verbose = 1, batch_size = [1000, 1200, 1400], epochs = [15,20,25])

# Define the parameter grid for hyperparameter tuning
param_dist = {
    'batch_size': [1000, 1200, 1400],
    'epochs': [15,20,25]
}

# Setting up RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, 
                                   n_iter=10, cv=3, verbose=2)

# Perform hyperparameter tuning using x_val and y_val
random_search_result = random_search.fit(x_train, y_train)

# Best parameters
best_params = random_search_result.best_params_
print(f"Best Parameters: {best_params}")

# Train the optimal model with best parameters
optimal_model = create_model()
optimal_model.fit(x_train, y_train, batch_size=best_params['batch_size'], epochs=best_params['epochs'])

/opt/conda/lib/python3.11/site-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 3 folds for each of 9 candidates, totalling 27 fits
Epoch 1/15
155/155 [==============================] - 2s 7ms/step - loss: 1.8920 - accuracy: 0.7298
Epoch 2/15
155/155 [==============================] - 1s 6ms/step - loss: 0.6727 - accuracy: 0.7816
Epoch 3/15
155/155 [==============================] - 1s 5ms/step - loss: 0.5427 - accuracy: 0.7967
Epoch 4/15
155/155 [==============================] - 1s 6ms/step - loss: 0.4688 - accuracy: 0.8315
Epoch 5/15
155/155 [==============================] - 1s 5ms/step - loss: 0.4114 - accuracy: 0.8602
Epoch 6/15
155/155 [==============================] - 1s 6ms/step - loss: 0.3788 - accuracy: 0.8807
Epoch 7/15
155/155 [==============================] - 1s 5ms/step - loss: 0.3690 - accuracy: 0.8814
Epoch 8/15
155/155 [==============================] - 1s 5ms/step - loss: 0.3451 - accuracy: 0.8842
Epoch 9/15
155/155 [==============================] - 1s 5ms/step - loss: 0.3283 - accuracy: 0.8839
Epoch 10/15
155/155 [===================

In [28]:
# nonscaled
calculate_performance_metrics(x_test, y_test, optimal_model)

3103/3103 [==============================] - 12s 4ms/step
Precision: 0.8851848930265782
Accuracy: 0.8763923859401752
F1 Score: 0.7204116444380958
AUC: 0.9753086118275315
Mean FNR: 0.2678411887269287
Mean TNR: 0.9680055954851158
Mean FPR: 0.03199440451488426
Mean TPR: 0.7321588112730714


In [22]:
# scaled
calculate_performance_metrics(x_test, y_test, optimal_model)

3103/3103 [==============================] - 2s 513us/step
Precision: 0.8687481071932716
Accuracy: 0.8576593816094269
F1 Score: 0.8527933888840304
AUC: 0.9531436375333651
Mean FNR: 0.352860377824012
Mean TNR: 0.9580964626777057
Mean FPR: 0.041903537322294376
Mean TPR: 0.6471396221759881
